In [1]:
import pandas as pd 

In [2]:
import numpy as np

In [3]:
from datetime import date   # para tener date.today()

In [4]:
import xlsxwriter # formato al xlsx exportado

In [6]:
stocks_1 = pd.read_excel("relaciones de stock (Mayor a 0) 17-10-22.xls")
stocks_1 = stocks_1[stocks_1.columns[~stocks_1.columns.isin(["Unnamed: 5","Unnamed: 7","Unnamed: 10"])]]
stocks_2 = pd.read_excel("relaciones de stock (Menor a 0) 17-10-22.xls")
mapping = {stocks_2.columns[0]:'familia'}
stocks_2 = stocks_2.rename(columns=mapping)
stocks = pd.concat([stocks_1, stocks_2]).reset_index(drop=True)
stocks

,familia,Artículo,Color,descripcion,fecha última venta,stock físico,cantidad de ventas,ventas MC,precio lista A,último costo MC
0,1.010 - ALFOMBRAS MOQUETTE,411,41514-41514,Moquette 401 Plus,NaN,NaN,NaN,NaN,NaN,NaN
1,1.010 - ALFOMBRAS MOQUETTE,411,41612-41612,Moquette 401 Plus,14/11/2019,25.62,0.00,0.00,26.0,11.810
2,1.010 - ALFOMBRAS MOQUETTE,COLU,24100-24100,Moquette Columbus ancho 4 metros,20/9/2022,119.60,74.40,"1,121.10",18.1,9.110
3,1.010 - ALFOMBRAS MOQUETTE,COLU,24101-24101,Moquette Columbus ancho 4 metros,13/9/2022,203.20,77.60,"1,756.01",18.1,9.110
4,1.010 - ALFOMBRAS MOQUETTE,COLU,24800-24800,Moquette Columbus ancho 4 metros,29/9/2022,124.00,160.00,"3,436.06",18.1,9.110
...,...,...,...,...,...,...,...,...,...,...
608,9.010 - LISTONES VINILICOS,LIS,K-6046-K-6046,"Contact Eco Wood 3mm capa uso 0.5, 3.32 m2",18/10/2022,-303.00,"1,170.00","76,440.06",80.7,34.85
609,9.099 - VARIOS ARTICULOS,ANG-AL,NaN,ANGULO DE ALUMINIO 15 MM,6/8/2022,-6.40,13.40,152.99,0.0,1.05
610,9.099 - VARIOS ARTICULOS,ENC-CIF,NaN,Encastre Sport Gym CIF,20/9/2022,"-10,000.00","10,000.00",0.00,0.0,0.0
611,9.099 - VARIOS ARTICULOS,GOMA-MOL,NaN,Goma Molida Premium bolsas 30 kg aprox,7/10/2022,-10.00,10.00,200.24,0.0,7.58


In [7]:
stocks['ventas MC'] = pd.to_numeric(stocks['ventas MC'], errors='coerce')
stocks['stock físico'] = pd.to_numeric(stocks['stock físico'], errors='coerce')
stocks['cantidad de ventas'] = pd.to_numeric(stocks['cantidad de ventas'], errors='coerce')
stocks['precio lista A'] = pd.to_numeric(stocks['precio lista A'], errors='coerce')
stocks['último costo MC'] = pd.to_numeric(stocks['último costo MC'], errors='coerce')
stocks.dtypes

familia                object
Artículo               object
Color                  object
descripcion            object
fecha última venta     object
stock físico          float64
cantidad de ventas    float64
ventas MC             float64
precio lista A        float64
último costo MC       float64
dtype: object

In [8]:
# stocks$`stock físico` <- ifelse(is.na(stocks$`stock físico`),0,stocks$`stock físico`)
stocks['stock físico'] = np.where(stocks['stock físico'].isna(),0,stocks['stock físico'] )

stocks['meses_stock'] = round(stocks['stock físico']/stocks['cantidad de ventas'],2)
stocks['meses_stock'] = np.where( (stocks['stock físico']==0) & (stocks['cantidad de ventas']==0) ,0,stocks['meses_stock'] )
# stocks.head(474)

stocks['meses_exceso'] = stocks['meses_stock']-12
stocks['meses_exceso'] = np.where( (stocks['stock físico']==0) & (stocks['cantidad de ventas']==0) ,0,stocks['meses_exceso'] )

stocks['stock_exceso_$'] = round(stocks['meses_exceso'] * stocks['último costo MC'] ,2)

stocks['si_exceso'] = np.where(stocks['meses_exceso'] > 0 , 1 , 0)

stocks = stocks.sort_values(by=['si_exceso','familia','Artículo','meses_exceso'], ascending=[False,True,True,False])

stocks

,familia,Artículo,Color,descripcion,fecha última venta,stock físico,cantidad de ventas,ventas MC,precio lista A,último costo MC,meses_stock,meses_exceso,stock_exceso_$,si_exceso
1,1.010 - ALFOMBRAS MOQUETTE,411,41612-41612,Moquette 401 Plus,14/11/2019,25.62,0.00,0.00,26.00,11.81,inf,inf,inf,1
12,1.010 - ALFOMBRAS MOQUETTE,FM,M-4919-M-4919,Fieltro Monaco Ancho 4.00,24/5/2022,394.00,6.00,117.74,17.04,8.39,65.67,53.67,450.29,1
13,1.010 - ALFOMBRAS MOQUETTE,FM,M-4975-M-4975,Fieltro Monaco Ancho 4.00,3/2/2022,35.20,2.64,44.47,17.04,8.39,13.33,1.33,11.16,1
15,1.010 - ALFOMBRAS MOQUETTE,GARD,16512-16512,Moquette Garden ancho 3.66,NaN,107.24,0.00,0.00,7.00,3.92,inf,inf,inf,1
16,1.010 - ALFOMBRAS MOQUETTE,GARD-N,16122-16122,Moquette Garden ancho 3.66,13/10/2022,853.89,49.59,596.64,9.50,5.18,17.22,5.22,27.04,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
575,9.099 - VARIOS ARTICULOS,REDU-AL,BRONCE-BRONCE,Reductor Aluminio,27/7/2022,43.00,12.00,163.00,0.00,5.73,3.58,-8.42,-48.25,0
576,9.099 - VARIOS ARTICULOS,REDU-AL,Plata-Plata,Reductor Aluminio,12/9/2022,4.00,7.00,79.32,0.00,6.70,0.57,-11.43,-76.58,0
612,9.099 - VARIOS ARTICULOS,TARU,NaN,Tarugo,19/9/2022,-190.00,190.00,0.00,0.00,0.08,-1.00,-13.00,-1.04,0
578,9.099 - VARIOS ARTICULOS,VAR-AL,NaN,Varillas de Aluminio,17/10/2022,216.00,NaN,NaN,6.00,1.44,NaN,NaN,NaN,0


In [9]:
# First you need to create a workbook:
# wb <- createWorkbook()
name2 = [format(date.today(), "%Y%m%d"), "stocks_exceso.xlsx"]
file_name = "-".join(name2)
sheet_name = "Summary"

writer = pd.ExcelWriter(file_name, engine='xlsxwriter')
stocks.to_excel(writer, sheet_name=sheet_name, startrow = 0, startcol=0, index = False)
writer.save()



color1="pink"
color2="papayawhip"
color3="seagreen1"
color4="seashell"
color5="slategray1"
color6="slategray3"
color7="tan"
color8="palegreen"
color9="palegoldenrod"
color10="paleturquoise1"
color11="lightyellow1"
color12="mintcream"
color13="mistyrose"
color14="mediumspringgreen"
color15="khaki"
color16="lightcyan1"
color17="lightblue1"
color18="greenyellow"
color19="lemonchiffon1"
color20="gray92"

np.unique(stocks['familia'])[0]

stocks['familia'][0]
f1 = wb.add_format({'font_color': 'blue', 'bold':True})
ws.conditional_format(
   'A1:N999',
   {
      'type':'formula', 'criteria':'$A2="1.010 - ALFOMBRAS MOQUETTE"', 'value':50, 'format':f1
   })
wb.close()

unique(stocks$familia)[1]
conditionalFormatting(wb, "stocks",
                      cols = 1:14,
                      rows = 2:999, rule = '$A2 == "1.010 - ALFOMBRAS MOQUETTE"', style = createStyle(bgFill = color1)
)
unique(stocks$familia)[2]
conditionalFormatting(wb, "stocks",
                      cols = 1:14,
                      rows = 2:999, rule = '$A2 == "1.011 - ALFOMBRA DIBUJADA"', style = createStyle(bgFill = color2)
)
unique(stocks$familia)[3]
conditionalFormatting(wb, "stocks",
                      cols = 1:14,
                      rows = 2:999, rule = '$A2 == "1.012 - ALFOMBRAS CAMINERAS"', style = createStyle(bgFill = color3)
)
unique(stocks$familia)[4]
conditionalFormatting(wb, "stocks",
                      cols = 1:14,
                      rows = 2:999, rule = '$A2 == "1.013 - ALFOMBRAS MODULARES"', style = createStyle(bgFill = color4)
)
unique(stocks$familia)[5]
conditionalFormatting(wb, "stocks",
                      cols = 1:14,
                      rows = 2:999, rule = '$A2 == "1.014 - ACCESORIOS  PISOS FLOTANTES"', style = createStyle(bgFill = color5)
)
unique(stocks$familia)[6]
conditionalFormatting(wb, "stocks",
                      cols = 1:14,
                      rows = 2:999, rule = '$A2 == "1.015 - PISOS FLOTANTES"', style = createStyle(bgFill = color6)
)
unique(stocks$familia)[7]
conditionalFormatting(wb, "stocks",
                      cols = 1:14,
                      rows = 2:999, rule = '$A2 == "1.016 - PISOS DEPORTIVOS"', style = createStyle(bgFill = color7)
)
unique(stocks$familia)[8]
conditionalFormatting(wb, "stocks",
                      cols = 1:14,
                      rows = 2:999, rule = '$A2 == "1.017 - SALDOS"', style = createStyle(bgFill = color8)
)
unique(stocks$familia)[9]
conditionalFormatting(wb, "stocks",
                      cols = 1:14,
                      rows = 2:999, rule = '$A2 == "1.018 - ALFOMBRA C/CUERO"', style = createStyle(bgFill = color9)
)
unique(stocks$familia)[10]
conditionalFormatting(wb, "stocks",
                      cols = 1:14,
                      rows = 2:999, rule = '$A2 == "1.019 - PANELES"', style = createStyle(bgFill = color10)
)
unique(stocks$familia)[11]
conditionalFormatting(wb, "stocks",
                      cols = 1:14,
                      rows = 2:999, rule = '$A2 == "1.110 - BALDOSAS VINILICAS"', style = createStyle(bgFill = color11)
)
unique(stocks$familia)[12]
conditionalFormatting(wb, "stocks",
                      cols = 1:14,
                      rows = 2:999, rule = '$A2 == "2.010 - FIELTROS"', style = createStyle(bgFill = color12)
)
unique(stocks$familia)[13]
conditionalFormatting(wb, "stocks",
                      cols = 1:14,
                      rows = 2:999, rule = '$A2 == "3.010 - CESPED"', style = createStyle(bgFill = color13)
)
unique(stocks$familia)[14]
conditionalFormatting(wb, "stocks",
                      cols = 1:14,
                      rows = 2:999, rule = '$A2 == "5.010 - CARPETAS Y FELPUDOS"', style = createStyle(bgFill = color14)
)
unique(stocks$familia)[15]
conditionalFormatting(wb, "stocks",
                      cols = 1:14,
                      rows = 2:999, rule = '$A2 == "6.009 - ACCESORIOS VINILICOS"', style = createStyle(bgFill = color15)
)
unique(stocks$familia)[16]
conditionalFormatting(wb, "stocks",
                      cols = 1:14,
                      rows = 2:999, rule = '$A2 == "6.010 - VINILICOS"', style = createStyle(bgFill = color16)
)
unique(stocks$familia)[17]
conditionalFormatting(wb, "stocks",
                      cols = 1:14,
                      rows = 2:999, rule = '$A2 == "9.010 - LISTONES VINILICOS"', style = createStyle(bgFill = color17)
)
unique(stocks$familia)[18]
conditionalFormatting(wb, "stocks",
                      cols = 1:14,
                      rows = 2:999, rule = '$A2 == "9.099 - VARIOS ARTICULOS"', style = createStyle(bgFill = color18)
)



conditionalFormatting(wb, "stocks",
                      cols = 1:14,
                      rows = 2:999, rule = '$N2 == 0', style = createStyle(bgFill = "white")
)


#rule = '$A2 == "1.010 - ALFOMBRAS MOQUETTE"'
#type = "beginsWith", rule = "A")
#type="contains", 
# rows = all_rows, rule = '$A2=="Merc 230"'


library(lubridate)
name2 <- paste(format(today(), "%Y%m%d"),"stocks_exceso.xlsx",sep="-")
saveWorkbook(wb, name2, overwrite = TRUE)